# K-Fold Cross Validation

### Load Data

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import KFold
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation, InputLayer
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
device_name = tf.test.gpu_device_name()
if len(device_name) > 0:
    print("Found GPU at: {}".format(device_name))
else:
    device_name = "/device:CPU:0"
    print("No GPU, using {}.".format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
! pip install -q kaggle

In [ ]:
from google.colab import files

files.upload() # upload your kaggle api key

In [ ]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d samuelcortinhas/muffin-vs-chihuahua-image-classification

100% 474M/474M [00:12<00:00, 42.7MB/s]
100% 474M/474M [00:12<00:00, 40.8MB/s]


In [ ]:
!unzip -q ./muffin-vs-chihuahua-image-classification.zip -d .

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount= True)

Mounted at /content/drive


In [ ]:
# Define the paths to the train and test directories
train_dir = '/content/train'
test_dir = '/content/test'

# Define image dimensions and batch size
img_height, img_width = 100, 100
batch_size = 32
color_mode='rgb'

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  train_dir,
  seed=56,
  image_size=(img_height, img_width),
  batch_size=batch_size,
  color_mode= color_mode)

Found 4733 files belonging to 2 classes.


In [ ]:
test_ds = tf.keras.utils.image_dataset_from_directory(
  test_dir,
  seed=56,
  image_size=(img_height, img_width),
  batch_size=batch_size,
  shuffle=False,
  color_mode= color_mode)

Found 1184 files belonging to 2 classes.


### Import functions

In [ ]:
from utilities import *

In [ ]:
train_images, train_labels = preprocess_train_dataset(train_ds)
test_images, test_labels = preprocess_test_dataset(test_ds)

In [ ]:
folds_accuracy= []
folds_loss = [] # zero-one loss

In [ ]:
kf = KFold(n_splits=5, shuffle = True, random_state = 56)

In [ ]:
es = EarlyStopping(monitor='val_accuracy', patience=15, restore_best_weights=True)

In [ ]:
with tf.device(device_name):
  for train_index, test_index in kf.split(train_images):
    # Subset the training data into training and validation sets
    Train_images_cv, Val_images_cv = train_images[train_index], train_images[test_index]
    Train_labels_cv, Val_labels_cv = train_labels[train_index], train_labels[test_index]

    model_cv = Sequential([
    InputLayer(input_shape=(100,100,1)),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((4, 4)),
    BatchNormalization(),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((4, 4)),
    BatchNormalization(),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((4, 4)),
    BatchNormalization(),
    Flatten(),
    Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    Dropout(0.7),
    BatchNormalization(),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])
    model_cv.compile(loss='binary_crossentropy',optimizer=keras.optimizers.Adam(0.001),metrics=['accuracy'])
    history_cv = model_cv.fit(Train_images_cv, Train_labels_cv, epochs=50, batch_size = 32, validation_data = (Val_images_cv, Val_labels_cv), callbacks=[es])

    loss,acc = model_cv.evaluate(Val_images_cv,Val_labels_cv)
    zero_one_loss = 1-acc
    folds_loss.append(zero_one_loss)
    folds_accuracy.append(acc)

    print("Accuracies per fold:", folds_accuracy)
    print("Zero-One Loss per fold:", folds_loss)
    average_accuracy = np.mean(folds_accuracy)
    print("Average Accuracy:", average_accuracy)
    average_loss = np.mean(folds_loss)
    print("Average Zero-One Loss:", average_loss)

Epoch 1/50
119/119 [==============================] - 12s 25ms/step - loss: 0.5941 - accuracy: 0.7200 - val_loss: 5.8747 - val_accuracy: 0.5480
Epoch 2/50
119/119 [==============================] - 2s 20ms/step - loss: 0.4980 - accuracy: 0.7863 - val_loss: 0.6400 - val_accuracy: 0.7233
Epoch 3/50
119/119 [==============================] - 2s 20ms/step - loss: 0.4372 - accuracy: 0.8291 - val_loss: 1.8077 - val_accuracy: 0.4984
Epoch 4/50
119/119 [==============================] - 2s 20ms/step - loss: 0.4920 - accuracy: 0.7871 - val_loss: 0.5375 - val_accuracy: 0.7487
Epoch 5/50
119/119 [==============================] - 2s 18ms/step - loss: 0.4155 - accuracy: 0.8368 - val_loss: 0.4751 - val_accuracy: 0.8215
Epoch 6/50
119/119 [==============================] - 2s 18ms/step - loss: 0.4048 - accuracy: 0.8465 - val_loss: 0.3837 - val_accuracy: 0.8395
Epoch 7/50
119/119 [==============================] - 2s 19ms/step - loss: 0.3778 - accuracy: 0.8574 - val_loss: 0.9269 - val_accuracy: 0.671